In [ ]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using HybridRobotDynamics
using SiLQR

In [ ]:
"""
Solver Setup
"""

# Bouncing ball with thrust model
system = bouncing_ball()

# Stage and terminal costs
Q = 1e-4 * diagm([1.0, 1.0, 0.0, 0.0])
R = 1e-6 * I(system.nu)
Qf = 1e4 * Q
stage(x, u) = x'*Q*x + u'*R*u
terminal(x) = x'*Qf*x

# RK4 integrator
rk4 = ExplicitIntegrator(:rk4)

# Problem parameters
N = 100
Δt = 0.01
params = SiLQR.Parameters(system, stage, terminal, rk4, N, Δt)

# Reference trajectory and initial conditions
xref = [10.0; 2.0; 0.0; 0.0]
uref = zeros(system.nu)
params.xrefs = [xref for k = 1:N]
params.urefs = [uref for k = 1:(N-1)]
params.x0 = [0.0, 2.0, 10.0, -2.0]
params.mI = :down

nothing

In [ ]:
"""
Solve using SiLQR
"""

# Solve
sol = SiLQR.Solution(params)
cache = SiLQR.Cache(params)
@time SiLQR.solve!(sol, cache, params; max_iter=1000)

# Visualize states
xs = reduce(vcat, sol.xs)
plot_2d_states(N, system.nx, (1,2), xs)

nothing